# Source

In [ ]:
# list all sources of ground truth files

# Wrangle

## concatenate ground truth and relevant metadata (see below)
### key metadata:
- datetime (YYYYMMDDHHMMSS)
- datetime (YYYYMMDDHHMMSS)
- presence/absence (1 or 0)
- easting (epsg:32611)
- northing (epsg:32611)
- latitude (epsg:4326)
- longitude (epsg:4326)

## convert to geopandas dataframe with metadata as features, point geometry in EPSG:32611

# Export

## save as geojson in data/vector/groundtruth.geojson
